In [4]:
import glob
import numpy as np
import sys
from numpy import asarray
np.set_printoptions(threshold=100)
import scipy.stats as st
import torch
import torchvision.transforms as T
from torchvision import transforms
from PIL import Image
import PIL
import matplotlib.pyplot as plt
from scipy.signal import convolve2d 
from torch.nn.quantized import functional as qF
from matplotlib.pyplot import imshow
%matplotlib inline
import cv2
import math
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

import os

In [5]:
#ideas for speeding up: flatten image and use array access instead of double for loops, time complexity from O(n^2) to O(n)
def get_dark_channel(im,sz):
    b,g,r = cv2.split(im)
    dc = cv2.min(cv2.min(r,g),b);
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(sz,sz))
    dark = cv2.erode(dc,kernel)
    return dark


def get_atmosphere(im, darkch, p):
    M, N = darkch.shape #get shape of dark channel
    flatI=im.flatten().reshape(409600,3)
    flatdark = darkch.flatten() #contiguously flatten the dark channel
    searchidx = (flatdark.argsort()[:round(M * N * p)])  # find top M * N * p indexes in the dark channel
    r=[]
    g=[]
    b=[]
    actualPixelIntensities=[flatI[searchidx] for index in searchidx]
    for intensity in actualPixelIntensities:
        r.append(intensity[0])
        g.append(intensity[1])
        b.append(intensity[2])
    return([np.max(b),np.max(g),np.max(r)])


def get_transmission(I, A, omega, w):
    map=(1 - omega * get_dark_channel(I / A, w))
    map=map*255
    map=map.astype(np.uint8)
    return(map) ## CVPR09, eq.12
    



def Recover(im,t,A,tx = .01):

    res = np.empty(im.shape,im.dtype);
    t = cv2.max(t,tx);
    

    for ind in range(0,3):
#         b,g,r=cv2.split(img)
#         b=(((b-A[0])/t) + A[0])
#         print(b)
#         g=(((g-A[1])/t) + A[1])
#         r=(((r-A[2])/t) + A[2])
#         b=np.uint8(b)
#         g=np.uint8(g)
#         r=np.uint8(r)
        res[:,:,ind] = ((im[:,:,ind]-A[ind])/t) + A[ind]
#         finalimage=cv2.merge((g,b,r))
        
    res=res*255
    res=res.astype(np.uint8)

    return res

def Guidedfilter(im,p,r,eps):
    mean_I = cv2.boxFilter(im,cv2.CV_64F,(r,r));
    mean_p = cv2.boxFilter(p, cv2.CV_64F,(r,r));
    mean_Ip = cv2.boxFilter(im*p,cv2.CV_64F,(r,r));
    cov_Ip = mean_Ip - mean_I*mean_p;

    mean_II = cv2.boxFilter(im*im,cv2.CV_64F,(r,r));
    var_I   = mean_II - mean_I*mean_I;

    a = cov_Ip/(var_I + eps);
    b = mean_p - a*mean_I;

    mean_a = cv2.boxFilter(a,cv2.CV_64F,(r,r));
    mean_b = cv2.boxFilter(b,cv2.CV_64F,(r,r));

    q = mean_a*im + mean_b;
    return q;

def TransmissionRefine(im,et):
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY);
    gray = np.float64(gray)/255;
    r = 60;
    eps = 0.0001;
    t = Guidedfilter(gray,et,r,eps);

    return t;


In [ ]:
img = mpimg.imread("pipes/train/frame00315_lrgb_jpg.rf.e413c82778d73310c963d6a6bea50b7c.jpg")

if img is None:
    print("NONE")
else:
#     I = img.astype('float64')/255;
    dark=get_dark_channel(img,3)
    print(type(dark))
#     darkimage=Image.fromarray(dark)
#     darkimage.save("darkchannel2.png")
#     image = Image.open("darkchannel2.png")
#     plt.imshow(image, cmap='gray')
    
#     a=get_atmosphere(img,get_dark_channel(img,3), .01)
#     t=get_transmission(img,a,.95,2)  ##smaller last value means higher resolution of image
#     tr = TransmissionRefine(img,t);
#     datat=Image.fromarray(tr)
#     datat = datat.convert('RGB')
#     datat.save("trasmission_map.png")


    
#     J=Recover(img,tr,a,.1)
#     Jt=Image.fromarray(J)
#     Jt = Jt.convert('RGB')
#     Jt.save("final.png")
# #     j=Image.open("final.png")
# #     plt.imshow(J)
#     cv2.imwrite("final.png",J*255);
#     cv2.imshow('J',J);
#     cv2.waitKey(0);




